In [5]:
__version__ = '0.3'
__author__  = "Robert Matern (r.matern@stud.uni-hannover.de)"
__date__    = ''
__url__     = ''
__copyright__ = "(C) 2015 Robert Matern"

In [6]:
# import pandas as pd
import numpy as np

#Dynamisches Programm

Die normale Modellformulierung des Auftragsannahmeproblems im Revenue Management von Instandhaltungsprozessen:

$$V(\textbf{c}, t) = \sum_{j \in \mathcal{J}}p_{j}(t)\max[V(\textbf{c}, t-1), r_{j} + V(\textbf{c}-\textbf{a}_j, t-1)] + p_{0}(t)V(\textbf{c}, t-1) $$

$$= V(\textbf{c}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t) \max[r_j - V(\textbf{c}, t-1) + V(\textbf{c}-\textbf{a}_j, t-1), 0]$$

## Algorithmus

In [7]:
solutions = {}

def DP(solutions, conditions, products, resources, capacities, consumtions, times):
    '''Berechnung der Erwartungswerte des Auftragsannahmeproblems.'''
    
    # Leere Wert für den Erwartungswert des aktuellen Systemzustands.
    value = 0
    
    #Aktueller Systemzustand wird generiert.
    capacity = capacities[1:]
    time = times[0]
    condition = np.append(capacity, times[0])
    
    #Aktueller Systemzustand wird in dem 'np.array' aller möglichen Systemzustände gesucht.
    state = np.where((conditions == condition).all(axis=1))[0][0]
    
    # Memofunktion: Das DP wird nur fortgeführt, sofern es nicht schon berechnet wurde.
    if state not in solutions:
                
        # Sofern es sich nicht um einen Endknoten des Entscheidungsbaums handelt,
        # werden folgende Schritte eingeleitet:
        if times[0]!=0:
            # Das DP(t-1) ohne Akeptanz wird gelöst und im Wert 'reject' gespeichert.
            capacity = capacities[1:]
            time = times[1]
            condition = np.append(capacity, times[0])
            state_reject = np.where((conditions == condition).all(axis=1))[0][0]
            if state_reject not in solutions:
                reject = DP(solutions, conditions, products, resources, capacities, consumtions, times[1:])
            else:
                reject = solutions[state_reject][1]
            # Für das DP(t-1) mit Akzeptanz wird ein Numpy-Array in der Länge der Anzahl an Produkten erstellt.
            accept = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            # For-Schleife über alle Produkte, sofern die Kapazitäten keinen negativen Werte annehmen.
            for j in products[1:]:
                change = capacities-consumtions[j]
                if np.all((change) >= 0):
                    # Initialisierung des DP(t-1) mit Akeptanz jeweils für ein Produkt j.
                    capacity = change[1:]
                    time = times[1]
                    condition = np.append(capacity, times[0])
                    state_accept = np.where((conditions == condition).all(axis=1))[0][0]
                    if state_accept not in solutions:
                        accept_j = DP(solutions, conditions, products, resources, change, consumtions, times[1:])
                        accept[j-1] = probs[j][times[0]]*max(revenues[j]-reject+accept_j, 0)
                    else:
                        accept[j-1] = solutions[state_accept][1]
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,t)=-∞, falls n[j] < 0.
                    accept[j-1] = 0
                            
            # Summierung des DP(t-1) ohne Akzeptanz sowie den Numpy-Array DP(t-1) mit Akzeptanz.
            value = np.around(reject + np.sum(accept), decimals=2)
            # Für den aktuellen Systemzustand wird der Ertragswert in das Dict "solutions" gespeichert.
            solutions[state] = [conditions[state], value]

        # Sofern es sich um einen Endknoten des Entscheidungsbaums handelt, werden folgende Schritte eingeleitet:
        else:
            # Erwartungswert enspricht der Grenzbedingung V(c,0)=0, für n >= 0.
            value = 0
            # Ein Endzustand wird mit einem Erwartungswert 0 in das Dict "solutions" gespeichert.
            solutions[state] = [conditions[state], value]
            return value        
            
    # Memofunktion: Sofern das Ergebnis breits berechnet wurde, wird der Wert aus dem Dict "solutions" verwendet.
    else:
        value = solutions[state][1]
                
    return value

## Erstellung der Struktur als NetworkX-Graph

In [8]:
import networkx as nx

def Structure(solutions, products, consumtions, revenues, probs):
    '''Generierung der Stuktur der Problemstellung.'''
    graph=nx.MultiDiGraph()

    for key in solutions.iterkeys():
        graph.add_node(key, label=solutions[key][0], value=solutions[key][1],
                   capacity=solutions[key][0][:-1], time=solutions[key][0][-1])
    graph.add_node("end")
    
    for i in solutions.iterkeys():
        if solutions[i][0][-1] == 0: # Ist der Zeitpunkt 0, dann verbinde diese Lösung mit dem 'end'-Knoten.
            graph.add_edge(i, "end", weight=0, revenue=0)
        else:
            for s in solutions.iterkeys():
                for j in products:
                    capa = []
                    capa = solutions[i][0][:-1] - consumtions[j][1:]
                    if np.array_equal(capa, solutions[s][0][:-1]) and solutions[i][0][-1]-1 == solutions[s][0][-1]:
                        if  j == 0:
                            #print i, s, revenues[j], 0
                            graph.add_edge(i, s, key=0, weight=0, weight_goal=solutions[s][1], revenue=revenues[j], goal=solutions[s][0], time=solutions[i][0][-1])
                        if  j != 0 and probs[j][solutions[i][0][-1]]>0:
                            for k in solutions.iterkeys():
                                if np.array_equal(solutions[i][0][:-1], solutions[k][0][:-1]) and solutions[i][0][-1]-1 == solutions[k][0][-1]:
                                    if revenues[j] >= solutions[k][1]-solutions[s][1]:
                                        graph.add_edge(i, s, key=j, weight=revenues[j]-solutions[k][1]+solutions[s][1], weight_goal=solutions[s][1], revenue=revenues[j], goal=solutions[s][0], time=solutions[i][0][-1])
                                    else:
                                        graph.add_edge(i, s, key=j, weight=0, weight_goal=solutions[s][1], revenue=revenues[j], goal=solutions[s][0], time=solutions[i][0][-1])

    return graph

## Ermittlung der optimalen Politik (Dijkstra Algorithmus)

In [9]:
def Best_Politic(graph, times, products, start=None):
    '''Ermittlung der optimalen Politik.'''
    
    print 'Optimalen Politik zum Zeitpunkt "t" und unter Beachtung der Restkapazitäten "c[h]":', '\n'
    
    # List mit topologischer Sortierung des Graphen wird erstellt.
    list = nx.topological_sort(graph)
    
    # Schleife über alle Knoten der topologisch sortierten Liste.
    for node in list:
        # Ist die Schleife zum Endknoten der Liste gekommen, dann wird sie unterbrochen.
        if node == 'end':
            break
        # Ist die Schleife zu einem Endzeitpunkt gekommen, dann wird sie übersprungen.
        if graph.node[node]['time'] == 0:
            continue
        # Sonst werden die möglichen Auftragsannahmen des Knotens in eine Liste geschrieben.
        else:
            array = []
            for suc in graph.successors(node):
                for order in graph.edge[node][suc].iterkeys():
                    array.append((node,suc,order,graph.edge[node][suc][order]['weight']))
                    
                    # Boolescher Wert 'not_best_politic' zur Ermittlung des Pfads wird angelegt.
                    graph.edge[node][suc][order]['not_best_politic']=True
            
            # Sofern keine Auftragsannahme möglich ist:
            if not array:
                print 'Zeitpunkt:', graph.node[node]['time'], 'Kapazität:', graph.node[node]['capacity']
                print '===> keine weitere Annahme einer Anfrage möglich. \n'
            # Sofern Auftragsannahmen möglich sind, wird die Option mit der besten Politik bestimmt.
            else:
                print 'Zeitpunkt:', graph.node[node]['time'], 'Kapazität:', graph.node[node]['capacity']
                maxweight = max(array, key=lambda x: x[3])
                print '===> Annahme des Auftrags:', maxweight[2]
                print 'Ertrag "r[j]" abzgl. Opportunitätskosten "OC[j,c,t]":', maxweight[3], '\n'
                
                # Boolescher Wert 'not_best_politic' für den Pfad wird bestimmt.
                graph.edge[maxweight[0]][maxweight[1]][maxweight[2]]['not_best_politic']=False
    
    # Optimaler Pfad wird mittels Dijkstra-Algorithmus geschrieben.
    path = nx.dijkstra_path(graph, source=list[0], target=list[-1], weight='not_best_politic')[:-1]
    print 'Optimale Politik:', path
    rev = [0] * len(times)
    for i, node in enumerate(path[:-1]):
        for order in graph.edge[node][path[i+1]].iterkeys():
            if graph.edge[node][path[i+1]][order]['not_best_politic'] == False:
                rev[graph.node[node]['time']] = graph.edge[node][path[i+1]][order]['revenue']
    print 'Revenue je Buchungsperiode:', rev[::-1]
    print 'Gesamt-Revenue:', sum(rev), '\n'
        
    return

In [10]:
def Best_Politic_Demand(graph, times, products, demands, start=None):
    '''Ermittlung der optimalen Politik.'''
    
    print 'Optimalen Politik zum Zeitpunkt "t" und unter Beachtung der Restkapazitäten "c[h]" sowie der tatsächlich eintreffenden Nachfrage "d[j,t]":', '\n'
    
    # List mit topologischer Sortierung des Graphen wird erstellt.
    list = nx.topological_sort(graph)
    
    # Schleife über alle Knoten der topologisch sortierten Liste.
    for node in list:
        # Ist die Schleife zum Endknoten der Liste gekommen, dann wird sie unterbrochen.
        if node == 'end':
            break
        # Ist die Schleife zu einem Endzeitpunkt gekommen, dann wird sie übersprungen.
        if graph.node[node]['time'] == 0:
            continue
        # Sonst werden die möglichen Auftragsannahmen des Knotens in eine Liste geschrieben.
        else:
            array = []
            for suc in graph.successors(node):
                for order in graph.edge[node][suc].iterkeys():
                    # Sofern keine Anfrage eintrifft, wird die mögliche Auftragsannahme nicht aufgenommen.
                    if demands[order][graph.edge[node][suc][order]['time']] == True:
                        array.append((node,suc,order,graph.edge[node][suc][order]['weight']))

                    # Boolescher Wert 'not_best_politic' zur Ermittlung des Pfads wird angelegt.
                    graph.edge[node][suc][order]['not_best_politic']=True
            
            # Sofern keine Auftragsannahme möglich ist:
            if not array:
                print 'Zeitpunkt:', graph.node[node]['time'], 'Kapazität:', graph.node[node]['capacity']
                print '===> keine weitere Annahme einer Anfrage möglich. \n'
            # Sofern Auftragsannahmen möglich sind, wird die Option mit der besten Politik bestimmt.
            else:
                print 'Zeitpunkt:', graph.node[node]['time'], 'Kapazität:', graph.node[node]['capacity']
                maxweight = max(array, key=lambda x: x[3])
                print '===> Annahme des Auftrags:', maxweight[2]
                print 'Ertrag "r[j]" abzgl. Opportunitätskosten "OC[j,c,t]":', maxweight[3], '\n'
                
                # Boolescher Wert 'not_best_politic' für den Pfad wird bestimmt.
                graph.edge[maxweight[0]][maxweight[1]][maxweight[2]]['not_best_politic']=False
    
    # Optimaler Pfad wird mittels Dijkstra-Algorithmus geschrieben.
    path = nx.dijkstra_path(graph, source=list[0], target=list[-1], weight='not_best_politic')[:-1]
    print 'Optimale Politik:', path
    rev = [0] * len(times)
    for i, node in enumerate(path[:-1]):
        for order in graph.edge[node][path[i+1]].iterkeys():
            if graph.edge[node][path[i+1]][order]['not_best_politic'] == False:
                rev[graph.node[node]['time']] = graph.edge[node][path[i+1]][order]['revenue']
    print 'Revenue je Buchungsperiode:', rev[::-1]
    print 'Gesamt-Revenue:', sum(rev), '\n'
        
    return

In [11]:
def Best_Path(graph, times, start=None):
    '''Ermittlung der optimalen Politik.'''
    list = nx.topological_sort(graph)
    if start == None:
        pol = nx.dijkstra_path(graph, list[0], "end", weight='weight_goal')
    else:
        pol = nx.dijkstra_path(graph, start, "end", weight='weight_goal')
    
    print "Kürzester Pfad:", pol[:-1], '\n'
    
    #### Funktioniert nicht, da er bei einem MultiDiGraph den Revenue überschreibt.
    #rev = [0] * len(times)
    #for i, node in enumerate(pol[:-1]):
    #    for order in graph.edge[node][pol[i+1]].iterkeys():
    #        rev[graph.node[node]['time']] = graph.edge[node][pol[i+1]][order]['revenue']
    #print 'Revenue je Buchungsperiode:', rev[::-1]
    #print 'Gesamt-Revenue:', sum(rev), '\n'
    
    return

In [12]:
import matplotlib.pyplot as plt
%matplotlib inline

def Drawing(graph, size_x=5, size_y=5):
    '''Grafische Darstellung des Entscheidungsbaums.'''
    draw = graph.copy()
    draw.remove_node('end')
    # Quelle: http://stackoverflow.com/a/11484144/4913569
    # ----
    # write dot file to use with graphviz
    # run "dot -Tpng test.dot >test.png"
    nx.write_dot(draw,'test.dot')

    # same layout using matplotlib with no labels
    pos=nx.graphviz_layout(draw,prog='dot')
    plt.figure(figsize=(size_x,size_y))
    plt.title("Entscheidungsbaum")
    nx.draw(draw,pos,with_labels=True, node_size=500)
    plt.savefig('nx_test.png')

    return